In [181]:
import openturns as ot
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from shapley import ShapleyIndices
from shapley.tests import ProductGaussian, AdditiveGaussian
from shapley.plots import set_style_paper, plot_sensitivity_results, plot_error, plot_var, plot_cover

%load_ext autoreload
%autoreload 2

%matplotlib inline

set_style_paper()

savefigs = True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [198]:
dim = 3
corr = 0.

beta = None
model = AdditiveGaussian(dim=dim, beta=beta)
model.margins = [ot.Normal()]*(dim-1) + [ot.Normal(0, 2.)]
theta = [0., 0., corr]
model.copula_parameters = theta
    
true_results = {
    'Shapley': model.shapley_indices,
    'First Sobol': model.first_sobol_indices,
    'Total Sobol': model.total_sobol_indices
}

In [211]:
n_var = 10000
n_inner = 3
n_outer = 1
n_boot = 1
n_run = 30
n_perms = 1000

all_shapley_results = np.zeros((n_run, dim, n_boot))
all_first_results = np.zeros((n_run, dim, n_boot))
all_total_results = np.zeros((n_run, dim, n_boot))

all_shapley_results_SE = np.zeros((n_run, dim))
all_first_results_SE = np.zeros((n_run, dim))
all_total_results_SE = np.zeros((n_run, dim))

for i_run in range(n_run):
    shapley = ShapleyIndices(model.input_distribution)
    shapley.build_sample(model=model, n_perms=n_perms, n_var=n_var, n_outer=int(n_outer), n_inner=int(n_inner))
    shapley_results = shapley.compute_indices(n_boot=n_boot)
    
    all_shapley_results[i_run] = shapley_results.full_shapley_indices
    all_first_results[i_run] = shapley_results.full_first_indices
    all_total_results[i_run] = shapley_results.full_total_indices

    all_shapley_results_SE[i_run] = shapley_results.shapley_indices_SE
    all_first_results_SE[i_run] = shapley_results.first_indices
    all_total_results_SE[i_run] = shapley_results.total_indices_SE
    print(i_run)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [212]:
from scipy import stats
ci_prob = 0.5
ci_method = 'tcl'
z_alpha = stats.norm.ppf((ci_prob)*0.5)

results = {
    'Shapley': all_shapley_results,
    'First Sobol': all_first_results,
    'Total Sobol': all_total_results
}
results_SE = {
    'Shapley': all_shapley_results_SE,
    'First Sobol': all_first_results_SE,
    'Total Sobol': all_total_results_SE
}

for name in results:
    result = results[name]
    no_boot_estimation = result[:, :, 0]
    result_SE = results_SE[name]
    true_indices = true_results[name]    
    if ci_method == 'tcl':
        ci_up = no_boot_estimation - z_alpha * result_SE
        ci_down = no_boot_estimation + z_alpha * result_SE
    elif ci_method == 'bootstrap':
        pass
    cover = ((ci_down < true_indices) & (ci_up > true_indices))
    error = (no_boot_estimation - true_indices).mean()
    print(name)
    print(cover.mean(axis=0), error, no_boot_estimation.mean(axis=0))

Shapley
[ 0.56666667  0.5         0.43333333] 3.42318765926e-17 [ 0.16057342  0.16916709  0.67025949]
First Sobol
[ 0.9         0.96666667  1.        ] 0.00165680611816 [ 0.16615105  0.17138105  0.66743832]
Total Sobol
[ 0.56666667  0.5         0.5       ] 0.00511892887164 [ 0.16945813  0.16764211  0.67825655]


In [ ]:
true_results